# Projeto 3 Ciência dos Dados - Mercado Imobiliário

Grupo 1 / 2C

Arthur Carvalho, Fernando Bichuette, Guilherme Lunetta, Rafael Kahn.

# É PRECISO INSTALAR A BIBLIOTECA GRAPHVIZ NO COMPUTADOR

## Pergunta

   Como podemos classificar imóveis da cidade de São Paulo por padrão de vida utilizando técnicas obtidas ao longo do       semestre? 

## Motivação/Introdução


   O mercado imobiliário é o mercado que trata de qualquer venda relacionada à propriedade e ao terreno no qual este está inserido. Dito isso, é razoável inferir que este é um dos grandes setores da economia de qualquer local desenvolvido. No contexto do Brasil, é inegável dizer que o lugar com maior potencial de liquidez do mercado imobiliário é São Paulo. Porém, em uma cidade de tal dimensão e com suas complexidades e características intrínsecas a si, é difícil fazer uma análise do mercado imobiliário apenas partindo do senso comum e de técnicas obsoletas. Assim, o grupo 1 do projeto 3 de ciência dos dados decide fazer uma análise exploratória e utilizar a técnica de regressão linear para estudar o mercado imobiliário de São Paulo. O resultado do projeto está disponível abaixo.



## Random Forest

   Random forest é uma técnica de classificação e regressão que utiliza de diversas árvores de escolha para formular uma decisão. Nesse método, a base de dados é separada em treinamento e teste, onde o treinamento utiliza uma amostra      aleatória de dados da base, nesse caso imóveis, para perceber padrões entre as variáveis atribuidas ao dado. Feito isso, o método é aplicado no teste, fazendo a mesma coisa que na base treinamento. Para o trabalho, será utilizado a parte da técnica relativa à classificação.

In [8]:
#Começar por importar as bibliotecas necessárias

%matplotlib inline
%reset -f
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import norm, uniform, probplot
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import seaborn as sns
import mpl_toolkits
from sklearn.tree import export_graphviz
import graphviz
from IPython.display import Image

ModuleNotFoundError: No module named 'graphviz'

In [ ]:
#Definir seed
RANDOM_SEED = 69
np.random.seed(RANDOM_SEED)


In [ ]:
#Abrindo a base de dados

data = pd.read_csv("sao-paulo-properties-april-2019.csv")
data.drop(data.columns[[15, 14]], axis=1, inplace=True)

# Limitar a análise para somente vendas

data = data[data['Negotiation Type'] == 'sale']
data.head()


In [ ]:
#Zonas de São Paulo

zona_sul = ['Campo Limpo', 'Capão Redondo', 'Vila Andrade', 'Cidade Dutra', 'Grajaú', 'Socorro', 'Cidade Ademar', 'Pedreira', 'Ipiranga', 'Sacomã', 'Jabaquara', "M'Boi Mirim", 'Jardim Ângela', 'Jardim Sao Luis', 'Parelheiros', 'Marsilac', 'Santo Amaro', 'Campo Belo', 'Campo Grande', 'Moema', 'Saúde', 'Vila Mariana', 'Brooklin', 'Vila Olimpia', 'Cursino', 'Jardim São Luis']
zona_leste = ['Aricanduva', 'Carrão', 'Vila Formosa', 'Cidade Tiradentes', 'Ermelino Matarazzo', 'Ponte Rasa', 'Guaianases', 'Lajeado', 'Itaim Paulista', 'Vila Curuça', 'Itaquera', 'Cidade Lider', 'José Bonifácio', 'Parque do Carmo', 'Mooca Água Rasa', 'Belém', 'Brás', 'Mooca', 'Pari', 'Tatuapé', 'Penha', 'Artur Alvim', 'Cangaíba', 'Penha', 'Vila Matilde', 'São Mateus', 'São Rafael', 'São Miguel', 'Jardim Helena', 'Vila Jacuí', 'Sapopemba', 'Vila Prudente', 'São Lucas', 'Água Rasa', 'Vila Curuçá', 'Cidade Líder', 'Guaianazes', 'Iguatemi'] 
zona_oeste = ['Butantã', 'Morumbi', 'Raposo Tavares', 'Rio Pequeno', 'Vila Sônia', 'Lapa', 'Barra Funda', 'Jaguara', 'Jaguaré', 'Perdizes', 'Vila Leopoldina', 'Pinheiros', 'Alto de Pinheiros', 'Itaim Bibi', 'Jardim Paulista', 'Pinheiros', 'Vila Madalena']
zona_norte = ['Casa Verde', 'Cachoeirinha', 'Limão', 'Brasilândia', 'Freguesia do Ó', 'Jaçanã', 'Tremembé', 'Perus', 'Anhanguera', 'Pirituba', 'Jaraguá', 'São Domingos', 'Santana', 'Tucuruvi', 'Mandaqui', 'Vila Maria', 'Vila Guilherme', 'Vila Medeiros', 'Medeiros']
centro = ['Sé Bela Vista', 'Bela Vista', 'Bom Retiro', 'Cambuci', 'Consolação', 'Liberdade', 'República', 'Santa Cecília', 'Sé']

In [ ]:
#Tirando /São Paulo dos bairros

filtro = data.District.str.replace('/São Paulo', '')
filtro
data.District = filtro


In [ ]:
#Adicionando zonas em função dos bairros para cada item do DataFrame

data.loc[(data.District).isin (zona_sul), 'Zone'] = 'ZS'
data.loc[(data.District).isin (zona_leste), 'Zone'] = 'ZL'
data.loc[(data.District).isin (zona_oeste), 'Zone'] = 'ZO'
data.loc[(data.District).isin (zona_norte), 'Zone'] = 'ZN'
data.loc[(data.District).isin (centro), 'Zone'] = 'Centro'


In [ ]:
#Novo data.head(), agora filtrado conforme as zonas da cidade

data.head()


In [ ]:
data.describe()


## Análise Explorativa

Vale lembrar que o target da análise é o preço do imóvel



### Price x Condo


In [ ]:
plt.scatter(data['Condo'][data['Condo']!=0], data['Price'][data['Condo']!=0]/1000000)
plt.xlabel('Preço do Condomínio')
plt.ylabel('Preço do imóvel (em milhões)')
plt.title('Preço Imóvel x Preço Condomínio')
plt.grid(True)
plt.show()


In [ ]:
data['Price'].corr(data['Condo'])

* Por esse gráfico podemos analisar que o preço do condomínio tem uma influência razoável no preço do imóvel. Existem outliers significantes que podreão ser desconsiderados. Analisando também a correlação, é possível dizer que o Preço do Condomínio é significante para nosso trabalho


### Price x Size

In [ ]:
a = data["Price"]/1000000

In [ ]:
x1 = data['Size']
y1 = a
plt.xlabel('Tamando do imóvel')
plt.ylabel('Preço do imóvel (em milhões)')
plt.title('Preço Imóvel x Tamanho do imóvel')
plt.scatter(x1,y1)
plt.grid(True)

In [ ]:
x1.corr(y1)

* A partir desse gráfico, pode-se observar que o tamanho do imóvel tem grande influência no preço do imóvel, com uma correlação de 0.82 e pouquíssimos outliers. Com certeza entra na lista de variáveis significantes


### Price x Toilets

In [ ]:
plt.scatter(data['Toilets'], data['Price']/1000000)
plt.xlabel('nº de Banheiros')
plt.ylabel('Preço do imóvel (em milhões)')
plt.title('Preço Imóvel x nº de Banheiros')
plt.grid(True)
plt.show()

In [ ]:
data['Price'].corr(data['Toilets'])

* O número de banheiros é razoavelmente significante em relação ao preço de um imóvel pois sua correlação está em torno de 0.6. Quase não tem outliers.


### Scatter Price x Rooms

In [ ]:
x2 =  data.loc[:,['Rooms']]
y2 =  a
plt.xlabel('Número de Quartos')
plt.ylabel('Preço do imóvel (em milhões)')
plt.title('Preço Imóvel x Número de quartos')
plt.scatter(x2,y2)
plt.grid(True)

In [ ]:
data['Price'].corr(data['Rooms'])

* Percebe-se que o preço tende a aumentar conforme aumentam os números de quartos. Uma possível explicação para o preço em apartamentos com 5 quartos ser menor é o fato de que estes podem estar em áreas onde o m² é menos valorizado, assim, apartamentos maiores poderão ter valores não tão altos. Isso também se mostra possível visto que a disparidade de preços em apartamentos com 3 e 4 quartos é muito grande. Porém, dito isso, a correlação ainda se mostra bem pequena, evidenciando o fato que há uma probabilidade de apartamentos com muitos quartos estarem em áreas menos valorizadas


### Boxplot Parking x Price


In [ ]:
# make boxplot with Seaborn
bplot=sns.boxplot(y=a, x='Parking', 
                 data=data, 
                 width=0.5,
                 palette="colorblind")
 
# add stripplot to boxplot with Seaborn
bplot=sns.stripplot(y=a, x='Parking', 
                   data=data, 
                   jitter=True, 
                   marker='o', 
                   alpha=0.5,
                   color='black')
plt.xlabel('Vagas no Estacionamento')
plt.ylabel('Preço do imóvel (em milhões)')
plt.title('Preço Imóvel x Vagas no Estacionamento')

In [ ]:
data["Parking"].corr(data["Price"])

* Boxplot mostrando a relação entre o preço e as vagas no estacionamento. Aqui vemos que, por mais que tem uma presença forte de outliers nos apartamentos com menos vagas, vale lembrar que estes ainda acatam por outliers por serem consideravelmente menos imóveis que apresentam essas características. Além disso, a correlação de 0.7 é forte, porém não o melhor indicador.


### Boxplot Elevator x Price

In [ ]:
data.Elevator = data.Elevator.astype('category')
data.Elevator.cat.categories = (['Sem', 'Com'])

print('Frequências absolutas por Elevador:')
ut1 = data.Elevator.value_counts(sort=False).reindex(['Sem', 'Com'])
print(ut1,'\n')

data.boxplot(column="Price",by="Elevator")
plt.xlabel('Elevador no prédio')
plt.ylabel('Preço do imóvel (em milhões)')
plt.title('Preço Imóvel x Elevador no prédio')
plt.show()


* Boxplot mostrando a relação entre o preço e se o imóvel possui ou não um elevador. Aqui vemos que, por mais que tem uma presença forte de outliers nos apartamentos com elevadores, vale lembrar que estes ainda acatam por outliers por serem consideravelmente menos imóveis que apresentam essas características.


### Scatterplot Suítes x Price

In [ ]:
xsuite = data["Suites"]
ysuite = a
plt.xlabel('Número de suítes')
plt.ylabel('Preço do imóvel (em milhões)')
plt.title('Preço Imóvel x Número de suítes')
plt.scatter(xsuite, ysuite)
plt.show()

In [ ]:
xsuite.corr(ysuite)

* Scatterplot mostrando a relação entre o número de suítes e o preço dos imóveis da base de dados estudada durante o projeto. Como pode-se perceber, conforme o número de suítes aumenta, o preço dos imóveis também aumentam.

## Classificador random forest

Já filtrada as variáveis obsoletas para o projeto, iremos utilizar da técnica de classificação random forest para encontrar padrões entre os imóveis listados

In [ ]:
data.Price.describe()

In [ ]:
data['Classification']='Muito Alto'
data.loc[(data.Price > 380000) & (data.Price <= 679000), "Classification"] = 'Alto' 
data.loc[(data.Price > 250000) & (data.Price <= 380000), "Classification"] = 'Médio' 
data.loc[(data.Price <= 250000), "Classification"] = 'Baixo' 

In [ ]:
data.Classification.value_counts(True)

In [ ]:
x = data[['Size', 'Condo', 'Rooms', 'Toilets', 'Suites', 'Swimming Pool']]
y = data['Classification']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=152)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(n_estimators=100)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print('Acurácia: {}'.format(accuracy_score(y_test, y_pred)))

In [10]:
matriz = pd.crosstab(y_test, y_pred, normalize = 'index')
matriz

NameError: name 'y_test' is not defined

## Matriz de Confusão

In [9]:
def plot_confusion_matrix(df_confusion, cmap=plt.cm.Blues):
    plt.matshow(df_confusion, cmap=cmap) # imshow
    plt.colorbar()
    tick_marks = np.arange(len(df_confusion.columns))
    plt.xticks(tick_marks, df_confusion.columns, rotation=45)
    plt.yticks(tick_marks, df_confusion.index)

plot_confusion_matrix(matriz)

NameError: name 'matriz' is not defined

* Como pode-se observar no gráfico da matriz de confusão plotado acima, o classificador tem uma eficácia muito boa ao obter resultados    **Verdadeiros-Verdadeiros** ( Quadrados de cor mais escura observado na diagonal do quadrado), principalmente ao classifcar imóveis categorizados como Baixo e Muito Alto. Além disso, a quantidade de **Falsos-Verdadeiros** obtidas também foi muito satisfatória, devido ao alto índice de acerto que pode ser observado nos quadrados brancos em volta da diagonal principal do gráfico. Em contra partida, embora exista classificações **Verdadeiro-Falso** ( Podendo ser  observados nos quadrados de cor mais acizentada nas bordas do gráfico), são números muitos pequenos sendo quase nulos (Quadrado Branco), confirmando assim que o classificador comete poucos erros.

## Árvore de decisão do classificador


In [ ]:
#Criando arquivo de imagem de uma única árvore do classificador:

# Escolhendo uma das árvores
estimator = model.estimators_[1]

# Transformando em imagem
imagem = export_graphviz(estimator, out_file=None, 
                feature_names = ['Size', 'Condo', 'Rooms', 'Toilets', 'Suites', 'Swimming Pool'],
                class_names = ['Muito Alto', 'Alto', 'Médio', 'Baixo'],
                rounded = True, proportion = False, 
                precision = 2, filled = True)
# Gerando PDF da imagem
garv = graphviz.Source(imagem)
garv.render(view = True)

* O código acima gera um pdf com a imagem de uma das árvores do classificador, para que se possa visualizar melhor o processo de como o método utilizado funciona e quais caminhos percorre para classificar um dos imóveis.

### Primeira imagem

<img src='Arvore_demonstracao.png' width='600'>

### Segunda imagem

<img src='Arvore_real.png' width='600'>

* A primeira imagem mostra uma árvore simplificada, para que seja possível se ter uma melhor visualização do processo. A segunda imagem é a árvore real, sem restrições de profundidade dos galhos da árvore ( a imagem está cortada devido ao fato de que a árvore é tão grande, que não é possível colocá-la inteira dentro de uma só imagem).

## Relevância das Respostas

Visto as acurácias relativas às quatro classes de imóveis em São Paulo, claramente podemos concluir que as classes baixo e muito alto se mostram mais bem analisadas. Visto isso, hipoteticamente falando, o nosso modelo pode ajudar pessoas de renda baixa e pessoas de muita alta renda a entenderem se um imóvel desejado têm sua faixa de preço justificada, de acordo com as especificações do imóvel. Em contrapartida, Para pessoas de média e alta renda, o modelo não é satisfatório.

## Tentativa de melhoria

   Ao pensarmos em como poderíamos melhorar nosso projeto, a única coisa que vem à cabeça é aumentar o número de variáveis exploratórias para quem sabe aumentar a acurácia do modelo. No entanto, algumas variáveis não foram utilizadas pois a partir de gráficos de dispersão e até mesmo boxplot's, foi possível perceber que não faziam diferença significante no resultado final, por isso essas variáveis não foram utilizadas. Para o futuro, a ideia é criar novas variáveis quantitativas e qualitativas para aperfeiçoar o modelo, de modo que a acurácia aumente para termos um modelo mais eficiente e preciso.

## Conclusão

   O trabalho de mercado imobiliário nos demonstrou eficiente para algumas classes, muito alto e baixo, e pouco eficiente para outras, notavelmente alto e médio. Visto que não há erros no código, a falha foi relativa ao próprio classificador.
   Para iterar ao trabalho, talvez seja uma boa ideia testar outro modelos. Isso possibilitaria ao menos comparar a eficiência dos modelos para tentar chegar em um consenso do melhor jeito de fazer um trabalho desse tipo. Além dessa possibilidade, também poderíamos ter vistos base de dados mais completas, com imóveis mais bem distribuídos pelas faixas de preços, para testarmos se a acurácia melhora.
   Em suma, o trabalho teve êxito em classificar parte dos imóveis de São Paulo, porém ainda peca em algumas áreas.

## Fontes:

https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py<br>
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html<br>
https://towardsdatascience.com/how-to-visualize-a-decision-tree-from-a-random-forest-in-python-using-scikit-learn-38ad2d75f21c